In [1]:
import os
import spyt
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pyspark.sql.functions as F 
import pyspark.sql.types as T 
from pyspark.sql.functions import col, lit, broadcast
from pyspark.sql.window import Window
from clan_tools.secrets.Vault import Vault
from clan_tools.data_adapters.YTAdapter import YTAdapter
os.environ['NUMEXPR_MAX_THREADS'] = '16'

pd.set_option('display.max.rows', 1000)
pd.set_option("display.max_columns", 250)
DEFAULT_SPARK_CONF = {
    "spark.executor.memory": "6G",
    "spark.executor.cores": 2,
    "spark.sql.session.timeZone": "UTC",
    "spark.dynamicAllocation.maxExecutors": 6,
    "spark.dynamicAllocation.enabled":True,
    "spark.sql.autoBroadcastJoinThreshold":-1,
    "spark.cores.min":12,
    "spark.driver.memory": "4G",
    "spark.executor.instances":6,
}

In [2]:
Vault().get_secrets()
yt_adapter = YTAdapter()
yt = yt_adapter.yt

spark = spyt.connect(spark_conf_args=DEFAULT_SPARK_CONF)
spyt.info(spark)

2021-08-28 20:26:58,908 - WARNING - spyt.conf - Your SPYT library has version 1.3.5 which is older than your cluster version 3.0.1-1.13.2+yandex. Some new features may not work as expected. Please update your cluster with spark-launch-yt utility
2021-08-28 20:27:18,503 - INFO - spyt.client - SPYT Cluster version: 3.0.1-1.13.2+yandex
2021-08-28 20:27:18,505 - INFO - spyt.client - SPYT library version: 1.3.5


# Dataset

### Features

In [3]:
# period to consider (including bounds)

period_start = '2021-04-01'
period_end = '2021-06-30'
numbers_of_days_to_consider = [1, 7, 14, 21, 30, 45]
rewrite_flag = True

In [4]:
TARGET_PATH = "//home/cloud_analytics/ml/scoring/trial_to_paid/tables/target"
FEATURES_YC_CONS = "//home/cloud_analytics/ml/scoring/trial_to_paid/tables/features_yc_cons"
FEATURES_VM_CUBE = "//home/cloud_analytics/ml/scoring/trial_to_paid/tables/features_vm_cube"

In [5]:
def max_by(x, y):
    return F.expr(f'max_by({x}, {y})')
def min_by(x, y):
    return F.expr(f'min_by({x}, {y})')

#### yc_consumption

In [6]:
yc_cons_pre = (
    spark.read
    .yt("//home/cloud-dwh/data/prod/cdm/dm_yc_consumption")
    .select(
        "billing_account_id",
        col("billing_record_msk_date"),
        col("billing_record_cost_rub"),
        col("billing_record_credit_rub"),
        col("billing_record_total_rub"),
        col("billing_account_usage_status"),
        col("billing_account_person_type"),
        col("billing_account_currency"),
        col("billing_account_state"),
        col("billing_account_is_fraud"),
        col("billing_account_is_isv"),
        col("billing_account_is_var"),
        (~col("crm_account_id").isNull()).alias("billing_account_is_crm_account"),
        col("crm_partner_manager"),
        col("crm_segment"),
        col("sku_lazy"),
        col("sku_name"),
        col("sku_service_group"),
        col("sku_service_name"),
        col("sku_subservice_name")
    )
)

yc_cons_pre.limit(5).toPandas()

,billing_account_id,billing_record_msk_date,billing_record_cost_rub,billing_record_credit_rub,billing_record_total_rub,billing_account_usage_status,billing_account_person_type,billing_account_currency,billing_account_state,billing_account_is_fraud,billing_account_is_isv,billing_account_is_var,billing_account_is_crm_account,crm_partner_manager,crm_segment,sku_lazy,sku_name,sku_service_group,sku_service_name,sku_subservice_name
0,dn2gikoq9dv4rmmkdkkm,2021-06-18,21.384000,0.0,21.384000,paid,individual,RUB,active,False,False,False,False,No Partner Manager,Mass,0.0,compute.vm.ram.v2,Infrastructure,compute,ram
1,dn2gikoq9dv4rmmkdkkm,2021-06-18,4.915462,0.0,4.915462,paid,individual,RUB,active,False,False,False,False,No Partner Manager,Mass,1.0,nbs.network-nvme.allocated,Infrastructure,compute,nbs_ssd
2,dn2gikoq9dv4rmmkdkkm,2021-06-18,0.000000,0.0,0.000000,paid,individual,RUB,active,False,False,False,False,No Partner Manager,Mass,0.0,network.ingress.inet,Infrastructure,cloud_network,ingress_inet
3,dn2gikoq9dv4rmmkdkkm,2021-06-18,0.000000,0.0,0.000000,paid,individual,RUB,active,False,False,False,False,No Partner Manager,Mass,0.0,network.egress.inet,Infrastructure,cloud_network,egress_inet
4,dn2gikoq9dv4rmmkdkkm,2021-06-18,0.894766,0.0,0.894766,paid,individual,RUB,active,False,False,False,False,No Partner Manager,Mass,1.0,nbs.network-hdd.allocated,Infrastructure,compute,nbs_hdd


In [7]:
ttt = (
    yc_cons_pre
    .groupby("billing_record_msk_date")
    .agg(F.sum("billing_record_total_rub").alias("billing_record_total_rub"))
    .sort("billing_record_msk_date")
    .toPandas()
)
ttt[926:1026]

,billing_record_msk_date,billing_record_total_rub
926,2021-04-01,9.138597e+06
927,2021-04-02,6.986576e+06
928,2021-04-03,6.706353e+06
929,2021-04-04,6.755470e+06
930,2021-04-05,7.092062e+06
931,2021-04-06,7.023458e+06
932,2021-04-07,7.136641e+06
933,2021-04-08,7.121027e+06
934,2021-04-09,7.196456e+06
935,2021-04-10,6.946717e+06


In [8]:
sku_name_cnts = yc_cons_pre[['sku_name']].groupby('sku_name').count().toPandas()
sku_name_cnts['ratio'] = sku_name_cnts['count']/sku_name_cnts['count'].sum()*100
sku_name_cnts = sku_name_cnts.sort_values('ratio', ascending=False).reset_index(drop=True)
sku_name_cnts['mostfreq_coverage'] = sku_name_cnts['ratio'].cumsum()

# 41 most frequent sku names covers 90% of all rows
N_mf_sku_nms = 41
names_mf_sku_nms = sku_name_cnts['sku_name'][:N_mf_sku_nms].tolist()
sku_name_cols = []

for name in names_mf_sku_nms:
    col_name = name.replace(' ', '_').replace('.', '_').replace('-', '_')
    sku_name_cols.append(
        F.sum(
            F.when(col("sku_name")==name, col("billing_record_total_rub"))
            .otherwise(lit(0))
        )
        .alias(f"sku_name_is_{col_name}")
    )

sku_name_cols.append(
    F.sum(
        F.when(~col("sku_name").isin(names_mf_sku_nms), col("billing_record_total_rub"))
        .otherwise(lit(0))
    )
    .alias("sku_name_is_other")
)

In [9]:
sku_group_cnts = yc_cons_pre[['sku_service_group']].groupby('sku_service_group').count().toPandas()
sku_group_cnts['ratio'] = sku_group_cnts['count']/sku_group_cnts['count'].sum()*100
sku_group_cnts = sku_group_cnts.sort_values('ratio', ascending=False).reset_index(drop=True)
sku_group_cnts['mostfreq_coverage'] = sku_group_cnts['ratio'].cumsum()

# we have only 8 sku groups so encode them all
names_all_sku_sgps = sku_group_cnts['sku_service_group'].tolist()
sku_group_cols = []

for name in names_all_sku_sgps:
    col_name = name.replace(' ', '_').replace('.', '_').replace('-', '_')
    sku_group_cols.append(
        F.sum(
            F.when(col("sku_service_group")==name, col("billing_record_total_rub"))
            .otherwise(lit(0))
        )
        .alias(f"sku_group_is_{col_name}")
    )

In [10]:
sku_service_cnts = yc_cons_pre[['sku_service_name']].groupby('sku_service_name').count().toPandas()
sku_service_cnts['ratio'] = sku_service_cnts['count']/sku_service_cnts['count'].sum()*100
sku_service_cnts = sku_service_cnts.sort_values('ratio', ascending=False).reset_index(drop=True)
sku_service_cnts['mostfreq_coverage'] = sku_service_cnts['ratio'].cumsum()

# we have only 20 sku services so encode them all
names_all_sku_srvs = sku_service_cnts['sku_service_name'].tolist()
sku_service_cols = []

for name in names_all_sku_srvs:
    col_name = name.replace(' ', '_').replace('.', '_').replace('-', '_')
    sku_service_cols.append(
        F.sum(
            F.when(col("sku_service_name")==name, col("billing_record_total_rub"))
            .otherwise(lit(0))
        )
        .alias(f"sku_service_is_{col_name}")
    )

In [11]:
sku_subservice_cnts = yc_cons_pre[['sku_subservice_name']].groupby('sku_subservice_name').count().toPandas()
sku_subservice_cnts['ratio'] = sku_subservice_cnts['count']/sku_subservice_cnts['count'].sum()*100
sku_subservice_cnts = sku_subservice_cnts.sort_values('ratio', ascending=False).reset_index(drop=True)
sku_subservice_cnts['mostfreq_coverage'] = sku_subservice_cnts['ratio'].cumsum()

# 24 most frequent sku subservices covers 95% of all rows
N_mf_sku_sbsrvs = 24
names_mf_sku_sbsrvs = sku_subservice_cnts['sku_subservice_name'][:24][:N_mf_sku_sbsrvs].tolist()
sku_subservice_cols = []

for name in names_mf_sku_sbsrvs:
    col_name = name.replace(' ', '_').replace('.', '_').replace('-', '_')
    sku_subservice_cols.append(
        F.sum(
            F.when(col("sku_subservice_name")==name, col("billing_record_total_rub"))
            .otherwise(lit(0))
        )
        .alias(f"sku_subservice_is_{col_name}")
    )

sku_subservice_cols.append(
    F.sum(
        F.when(~col("sku_subservice_name").isin(names_mf_sku_sbsrvs), col("billing_record_total_rub"))
        .otherwise(lit(0))
    )
    .alias("sku_subservice_name_is_other")
)

In [12]:
# yc consumption features
yc_cons = (
    yc_cons_pre
    .groupby("billing_account_id", "billing_record_msk_date")
    .agg(
        F.sum("billing_record_cost_rub").alias("billing_record_cost_rub"),
        F.sum("billing_record_credit_rub").alias("billing_record_credit_rub"),
        F.sum("billing_record_total_rub").alias("billing_record_total_rub"),
        max_by("billing_account_usage_status", "billing_record_msk_date")
            .alias("billing_account_usage_status"),
        max_by("billing_account_person_type", "billing_record_msk_date").alias("billing_account_person_type"),
        max_by("billing_account_currency", "billing_record_msk_date").alias("billing_account_currency"),
        max_by("billing_account_state", "billing_record_msk_date").alias("billing_account_state"),
        (F.sum(col("billing_account_is_fraud").cast("int"))>0).alias("billing_account_is_fraud"),
        (F.sum(col("billing_account_is_isv").cast("int"))>0).alias("billing_account_is_isv"),
        (F.sum(col("billing_account_is_var").cast("int"))>0).alias("billing_account_is_var"),
        (F.sum(col("billing_account_is_crm_account").cast("int"))>0).alias("billing_account_is_crm_account"),
        max_by("crm_partner_manager", "billing_record_msk_date").alias("crm_partner_manager"),
        max_by("crm_segment", "billing_record_msk_date").alias("crm_segment"),
        (F.sum(col("sku_lazy"))>0).alias("sku_lazy"),
        *sku_name_cols,
        *sku_group_cols,
        *sku_service_cols,
        *sku_subservice_cols,
    )
)

yc_cons.limit(5).toPandas()

,billing_account_id,billing_record_msk_date,billing_record_cost_rub,billing_record_credit_rub,billing_record_total_rub,billing_account_usage_status,billing_account_person_type,billing_account_currency,billing_account_state,billing_account_is_fraud,billing_account_is_isv,billing_account_is_var,billing_account_is_crm_account,crm_partner_manager,crm_segment,sku_lazy,sku_name_is_nbs_network_hdd_allocated,sku_name_is_network_public_fips,sku_name_is_network_egress_inet,sku_name_is_network_ingress_inet,sku_name_is_nbs_network_nvme_allocated,sku_name_is_compute_vm_ram_v2,sku_name_is_network_public_fips_deallocated,sku_name_is_storage_bucket_used_space_standard,sku_name_is_compute_snapshot,sku_name_is_compute_vm_cpu_c05_v2,sku_name_is_compute_vm_cpu_c100_v2,sku_name_is_storage_api_network_inet_egress,sku_name_is_compute_vm_ram,sku_name_is_storage_api_get_standard,sku_name_is_storage_bucket_used_space_cold,sku_name_is_compute_vm_cpu_50_v2,sku_name_is_compute_vm_cpu_c20_v2,sku_name_is_storage_api_network_inet_ingress,sku_name_is_storage_api_network_cloud_egress,sku_name_is_storage_api_put_standard,sku_name_is_compute_vm_cpu_c100,sku_name_is_compute_vm_cpu_c05,sku_name_is_storage_api_head_standard,sku_name_is_compute_image,sku_name_is_marketplace_windows_cpu_c100,sku_name_is_mdb_cluster_network_nvme_pg,sku_name_is_cr_bucket_used_space_standard,sku_name_is_storage_api_network_cloud_ingress,sku_name_is_support_standard_fixed_consumption_v1,sku_name_is_storage_api_delete,sku_name_is_storage_api_put_cold,sku_name_is_mdb_cluster_pg_v2_ram,sku_name_is_network_ingress_inet_antiddos_qrator,sku_name_is_nlb_balancer_active,sku_name_is_network_public_fips_lb,sku_name_is_nlb_vip_bytes_ingress,sku_name_is_marketplace_windows_cpu_c05,sku_name_is_compute_vm_ram_preemptible_v2,sku_name_is_storage_api_post_cold,sku_name_is_mdb_cluster_network_nvme_mysql,sku_name_is_mdb_cluster_network_hdd_pg,sku_name_is_other,sku_group_is_Infrastructure,sku_group_is_Data_Storage_and_Analytics,sku_group_is_Marketplace,sku_group_is_Support,sku_group_is_Cloud_Native,sku_group_is_ML_and_AI,sku_group_is_Adjustments,sku_group_is_Tracker,sku_service_is_compute,sku_service_is_cloud_network,sku_service_is_storage,sku_service_is_mdb,sku_service_is_marketplace,sku_service_is_support,sku_service_is_cloud_ai,sku_service_is_nlb,sku_service_is_cr,sku_service_is_serverless,sku_service_is_kms,sku_service_is_mk8s,sku_service_is_ydb,sku_service_is_ymq,sku_service_is_monitoring,sku_service_is_api_gateway,sku_service_is_iot,sku_service_is_adjustments,sku_service_is_tracker,sku_service_is_datalens,sku_service_is_dns,sku_subservice_is_nbs_hdd,sku_subservice_is_fips,sku_subservice_is_cpu,sku_subservice_is_ram,sku_subservice_is_egress_inet,sku_subservice_is_ingress_inet,sku_subservice_is_used_space,sku_subservice_is_nbs_ssd,sku_subservice_is_storage,sku_subservice_is_postgres,sku_subservice_is_snapshot,sku_subservice_is_get,sku_subservice_is_put,sku_subservice_is_windows,sku_subservice_is_mysql,sku_subservice_is_head,sku_subservice_is_standard,sku_subservice_is_mongo,sku_subservice_is_clickhouse,sku_subservice_is_speech,sku_subservice_is_image,sku_subservice_is_delete,sku_subservice_is_redis,sku_subservice_is_nlb,sku_subservice_name_is_other
0,dn200017upuhslibj3i4,2021-05-20,17.969556,-17.969556,0.0,trial,individual,RUB,suspended,True,False,False,False,No Partner Manager,Mass,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dn200017upuhslibj3i4,2021-06-22,17.867717,-17.867717,0.0,trial,individual,RUB,suspended,True,False,False,False,No Partner Manager,Mass,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [13]:
ttt = (
    yc_cons
    .groupby("billing_record_msk_date")
    .agg(F.sum("billing_record_total_rub").alias("billing_record_total_rub"))
    .sort("billing_record_msk_date")
    .toPandas()
)
ttt[926:1026]

,billing_record_msk_date,billing_record_total_rub
926,2021-04-01,9.138597e+06
927,2021-04-02,6.986576e+06
928,2021-04-03,6.706353e+06
929,2021-04-04,6.755470e+06
930,2021-04-05,7.092062e+06
931,2021-04-06,7.023458e+06
932,2021-04-07,7.136641e+06
933,2021-04-08,7.121027e+06
934,2021-04-09,7.196456e+06
935,2021-04-10,6.946717e+06


#### grid

In [14]:
# billing_account_id with all dates (as an event_date column) 
#           starting from max of its first mention or period_start

date_grid = (
    spark
    .createDataFrame(pd.DataFrame({"event_date": pd.date_range(period_start, period_end)}))
    .select(col("event_date").cast("date"))
)

all_ba_grid = (
    yc_cons
    .groupby("billing_account_id")
    .agg(F.min("billing_record_msk_date").alias("start_date"), 
         F.sum(col("billing_account_is_fraud").cast("int")).alias("is_fraud"), 
         min_by("billing_account_usage_status", "billing_record_msk_date").alias("ba_usage_status"))
    .filter(col("is_fraud")==0)
    .filter(col("ba_usage_status")!="service")
    .join(date_grid, on=(col("start_date")<=col("event_date")), how="cross")
    .select("billing_account_id", col("event_date").alias("billing_record_msk_date"))
)

#### yc_cons_full

In [15]:
colnames_yc_cons = yc_cons.limit(0).toPandas().columns.tolist()

def sparkNVL(colname, replace_val):
    return F.when(col(colname).isNull(), lit(replace_val)).otherwise(col(colname)).alias(colname)

yc_cons_full = (
    all_ba_grid
    .join(yc_cons, on=["billing_account_id", "billing_record_msk_date"], how="left")
    .select(
        "billing_account_id",
        F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd").alias("billing_record_msk_date"),
        sparkNVL("billing_record_cost_rub", 0),
        sparkNVL("billing_record_credit_rub", 0),
        sparkNVL("billing_record_total_rub", 0),
        "billing_account_usage_status",
        "billing_account_person_type",
        "billing_account_currency",
        "billing_account_state",
        "billing_account_is_fraud",
        "billing_account_is_isv",
        "billing_account_is_var",
        "billing_account_is_crm_account",
        "crm_partner_manager",
        "crm_segment",
        col("sku_lazy").cast("int"),
        *[sparkNVL(colname, 0) for colname in colnames_yc_cons[16:]]
    )
)

yc_cons_full.limit(5).toPandas()

,billing_account_id,billing_record_msk_date,billing_record_cost_rub,billing_record_credit_rub,billing_record_total_rub,billing_account_usage_status,billing_account_person_type,billing_account_currency,billing_account_state,billing_account_is_fraud,billing_account_is_isv,billing_account_is_var,billing_account_is_crm_account,crm_partner_manager,crm_segment,sku_lazy,sku_name_is_nbs_network_hdd_allocated,sku_name_is_network_public_fips,sku_name_is_network_egress_inet,sku_name_is_network_ingress_inet,sku_name_is_nbs_network_nvme_allocated,sku_name_is_compute_vm_ram_v2,sku_name_is_network_public_fips_deallocated,sku_name_is_storage_bucket_used_space_standard,sku_name_is_compute_snapshot,sku_name_is_compute_vm_cpu_c05_v2,sku_name_is_compute_vm_cpu_c100_v2,sku_name_is_storage_api_network_inet_egress,sku_name_is_compute_vm_ram,sku_name_is_storage_api_get_standard,sku_name_is_storage_bucket_used_space_cold,sku_name_is_compute_vm_cpu_50_v2,sku_name_is_compute_vm_cpu_c20_v2,sku_name_is_storage_api_network_inet_ingress,sku_name_is_storage_api_network_cloud_egress,sku_name_is_storage_api_put_standard,sku_name_is_compute_vm_cpu_c100,sku_name_is_compute_vm_cpu_c05,sku_name_is_storage_api_head_standard,sku_name_is_compute_image,sku_name_is_marketplace_windows_cpu_c100,sku_name_is_mdb_cluster_network_nvme_pg,sku_name_is_cr_bucket_used_space_standard,sku_name_is_storage_api_network_cloud_ingress,sku_name_is_support_standard_fixed_consumption_v1,sku_name_is_storage_api_delete,sku_name_is_storage_api_put_cold,sku_name_is_mdb_cluster_pg_v2_ram,sku_name_is_network_ingress_inet_antiddos_qrator,sku_name_is_nlb_balancer_active,sku_name_is_network_public_fips_lb,sku_name_is_nlb_vip_bytes_ingress,sku_name_is_marketplace_windows_cpu_c05,sku_name_is_compute_vm_ram_preemptible_v2,sku_name_is_storage_api_post_cold,sku_name_is_mdb_cluster_network_nvme_mysql,sku_name_is_mdb_cluster_network_hdd_pg,sku_name_is_other,sku_group_is_Infrastructure,sku_group_is_Data_Storage_and_Analytics,sku_group_is_Marketplace,sku_group_is_Support,sku_group_is_Cloud_Native,sku_group_is_ML_and_AI,sku_group_is_Adjustments,sku_group_is_Tracker,sku_service_is_compute,sku_service_is_cloud_network,sku_service_is_storage,sku_service_is_mdb,sku_service_is_marketplace,sku_service_is_support,sku_service_is_cloud_ai,sku_service_is_nlb,sku_service_is_cr,sku_service_is_serverless,sku_service_is_kms,sku_service_is_mk8s,sku_service_is_ydb,sku_service_is_ymq,sku_service_is_monitoring,sku_service_is_api_gateway,sku_service_is_iot,sku_service_is_adjustments,sku_service_is_tracker,sku_service_is_datalens,sku_service_is_dns,sku_subservice_is_nbs_hdd,sku_subservice_is_fips,sku_subservice_is_cpu,sku_subservice_is_ram,sku_subservice_is_egress_inet,sku_subservice_is_ingress_inet,sku_subservice_is_used_space,sku_subservice_is_nbs_ssd,sku_subservice_is_storage,sku_subservice_is_postgres,sku_subservice_is_snapshot,sku_subservice_is_get,sku_subservice_is_put,sku_subservice_is_windows,sku_subservice_is_mysql,sku_subservice_is_head,sku_subservice_is_standard,sku_subservice_is_mongo,sku_subservice_is_clickhouse,sku_subservice_is_speech,sku_subservice_is_image,sku_subservice_is_delete,sku_subservice_is_redis,sku_subservice_is_nlb,sku_subservice_name_is_other
0,dn2000o2g9kv69el386u,2021-04-01,0.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dn2000o2g9kv69el386u,2021-05-14,0.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [16]:
ttt = (
    yc_cons_full
    .groupby("billing_record_msk_date")
    .agg(F.sum("billing_record_total_rub").alias("billing_record_total_rub"))
    .sort("billing_record_msk_date")
    .toPandas()
)
ttt[:100]

,billing_record_msk_date,billing_record_total_rub
0,2021-04-01,6.260137e+06
1,2021-04-02,4.699875e+06
2,2021-04-03,4.485682e+06
3,2021-04-04,4.547304e+06
4,2021-04-05,4.739249e+06
5,2021-04-06,4.735596e+06
6,2021-04-07,4.857094e+06
7,2021-04-08,4.840534e+06
8,2021-04-09,4.893871e+06
9,2021-04-10,4.661356e+06


In [17]:
if rewrite_flag:
    _ = (
        yc_cons_full
        .write
        .yt(FEATURES_YC_CONS, mode="overwrite")
    )

#### vm_cube

In [18]:
vm_cube = (
    spark.read
    .yt("//home/cloud_analytics/compute_logs/vm_cube/vm_cube")
    .groupby("ba_id", F.to_date(F.from_unixtime("slice_time")).alias("billing_record_msk_date"), "vm_id")
    .agg(F.max("vm_cores").alias("cores"),
         F.max("vm_gpus").alias("gpus"),
         F.max("vm_memory").alias("memory"),
         F.max("vm_age_days").alias("vm_age_days"))
    .select(["billing_record_msk_date", "vm_id", "cores", "gpus", "memory", 
             "vm_age_days", col("ba_id").alias("billing_account_id")])
    .groupby("billing_account_id", "billing_record_msk_date")
    .agg(
        F.sum("cores").alias("vm_cores_sum"),
        F.mean("cores").alias("vm_cores_avg"),
        F.min("cores").alias("vm_cores_min"),
        F.max("cores").alias("vm_cores_max"),
        F.sum("gpus").alias("vm_gpus_sum"),
        F.mean("gpus").alias("vm_gpus_avg"),
        F.min("gpus").alias("vm_gpus_min"),
        F.max("gpus").alias("vm_gpus_max"),
        F.sum("memory").alias("vm_memory_sum"),
        F.mean("memory").alias("vm_memory_avg"),
        F.min("memory").alias("vm_memory_min"),
        F.max("memory").alias("vm_memory_max"),
        F.sum("vm_age_days").alias("vm_age_days_sum"),
        F.mean("vm_age_days").alias("vm_age_days_avg"),
        F.min("vm_age_days").alias("vm_age_days_min"),
        F.max("vm_age_days").alias("vm_age_days_max"),
        F.count("vm_id").alias("vm_count")
    )
)

vm_cube.limit(5).toPandas()

,billing_account_id,billing_record_msk_date,vm_cores_sum,vm_cores_avg,vm_cores_min,vm_cores_max,vm_gpus_sum,vm_gpus_avg,vm_gpus_min,vm_gpus_max,vm_memory_sum,vm_memory_avg,vm_memory_min,vm_memory_max,vm_age_days_sum,vm_age_days_avg,vm_age_days_min,vm_age_days_max,vm_count
0,dn2itpredkksnb3k00v6,2021-08-11,58.0,2.636364,2.0,8.0,0,0.000000,0,0,79.5,3.613636,0.5,8.0,273.0,12.409091,6.0,13.0,22
1,dn2k93c6l0l9it3qq9k4,2021-03-02,128.0,4.000000,4.0,4.0,0,0.000000,0,0,128.0,4.000000,4.0,4.0,0.0,0.000000,0.0,0.0,32
2,dn2b14f026i7l5emumhr,2021-08-07,6.0,2.000000,2.0,2.0,0,0.000000,0,0,1.5,0.500000,0.5,0.5,0.0,0.000000,0.0,0.0,3
3,dn2char50jflpgthkm57,2019-04-17,6566.0,2.746131,1.0,32.0,0,0.000000,0,0,21722.0,9.084902,0.5,256.0,43257.0,18.091593,0.0,59.0,2391
4,dn2char50jflpgthkm57,2021-05-04,61845.0,3.884736,1.0,80.0,537,0.033731,0,4,271831.0,17.074812,0.5,640.0,488369.0,30.676445,0.0,442.0,15920


#### vm_cube_full

In [19]:
colnames_vm_cube = vm_cube.limit(0).toPandas().columns.tolist()

vm_cube_full = (
    all_ba_grid
    .join(vm_cube, on=["billing_account_id", "billing_record_msk_date"], how="left")
    .select(
        "billing_account_id",
        F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd").alias("billing_record_msk_date"),
        *[sparkNVL(colname, 0) for colname in colnames_vm_cube[2:]]
    )
)

vm_cube_full.limit(5).toPandas()

,billing_account_id,billing_record_msk_date,vm_cores_sum,vm_cores_avg,vm_cores_min,vm_cores_max,vm_gpus_sum,vm_gpus_avg,vm_gpus_min,vm_gpus_max,vm_memory_sum,vm_memory_avg,vm_memory_min,vm_memory_max,vm_age_days_sum,vm_age_days_avg,vm_age_days_min,vm_age_days_max,vm_count
0,dn2000o2g9kv69el386u,2021-04-01,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,dn2000o2g9kv69el386u,2021-04-13,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,dn2000o2g9kv69el386u,2021-04-18,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,dn2000o2g9kv69el386u,2021-05-05,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,dn2000o2g9kv69el386u,2021-05-07,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [20]:
if rewrite_flag:
    _ = (
        vm_cube_full
        .write
        .yt(FEATURES_VM_CUBE, mode="overwrite")
    )

#### target

In [21]:
target_pre = (
    spark
    .read
    .yt(FEATURES_YC_CONS)
    .select(
        "billing_account_id",
        F.to_date("billing_record_msk_date").alias("billing_record_msk_date"),
        "billing_record_cost_rub",
        "billing_record_total_rub"
    )
)

In [22]:
ttt = (
    target_pre
    .groupby("billing_record_msk_date")
    .agg(F.sum("billing_record_total_rub").alias("billing_record_total_rub"))
    .sort("billing_record_msk_date")
    .toPandas()
)
ttt[:100]

,billing_record_msk_date,billing_record_total_rub
0,2021-04-01,6.260137e+06
1,2021-04-02,4.699875e+06
2,2021-04-03,4.485682e+06
3,2021-04-04,4.547304e+06
4,2021-04-05,4.739249e+06
5,2021-04-06,4.735596e+06
6,2021-04-07,4.857094e+06
7,2021-04-08,4.840534e+06
8,2021-04-09,4.893871e+06
9,2021-04-10,4.661356e+06


In [23]:
# calculate targets
def date_to_bigint(dt_col):
    return F.datediff(dt_col, lit("1900-01-01")).cast("bigint")

def make_window(N):
    if (N == 0):
        raise ValueError("N must be greater or less than 0")
    if N > 0:
        w = (
            Window
            .partitionBy("billing_account_id")
            .orderBy(date_to_bigint("billing_record_msk_date"))
            .rangeBetween(1, N)
        )
    else:
        w = (
            Window
            .partitionBy("billing_account_id")
            .orderBy(date_to_bigint("billing_record_msk_date"))
            .rangeBetween(N, -1)
        )
    return w

def next_N_days_cons(N):
    ans = F.sum("billing_record_total_rub").over(make_window(N))
    w = Window.partitionBy("billing_account_id")
    cond = (F.datediff(F.max("billing_record_msk_date").over(w), col("billing_record_msk_date"))>=N)
    colname = f"prev_{abs(N)}d_cons" if (N<0) else f"next_{N}d_cons"
    return F.when(cond, ans).otherwise(lit(None)).alias(colname)

target = (
    target_pre
    .select(
        "billing_account_id",
        "billing_record_msk_date",
        F.sum(
            F.when(col("billing_record_cost_rub")==0, lit(1))
            .otherwise(lit(0))
        ).over(make_window(Window.unboundedPreceding)).alias("days_not_used"),
        *[next_N_days_cons(-N) for N in numbers_of_days_to_consider],
        *[next_N_days_cons(N) for N in numbers_of_days_to_consider]
    )
)

target.limit(5).toPandas()

,billing_account_id,billing_record_msk_date,days_not_used,prev_1d_cons,prev_7d_cons,prev_14d_cons,prev_21d_cons,prev_30d_cons,prev_45d_cons,next_1d_cons,next_7d_cons,next_14d_cons,next_21d_cons,next_30d_cons,next_45d_cons
0,dn2003uvl006bf282eob,2021-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,dn2003uvl006bf282eob,2021-04-02,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dn2003uvl006bf282eob,2021-04-03,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dn2003uvl006bf282eob,2021-04-04,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dn2003uvl006bf282eob,2021-04-05,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
if rewrite_flag:
    _ = (
        target
        .withColumn("billing_record_msk_date", F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd"))
        .write
        .yt(TARGET_PATH, mode="overwrite")
    )

#### Reset spark

In [25]:
spyt.stop(spark)

spark = spyt.connect(spark_conf_args=DEFAULT_SPARK_CONF)
spyt.info(spark)

2021-08-28 20:59:19,700 - WARNING - spyt.conf - Your SPYT library has version 1.3.5 which is older than your cluster version 3.0.1-1.13.2+yandex. Some new features may not work as expected. Please update your cluster with spark-launch-yt utility
2021-08-28 20:59:34,939 - INFO - spyt.client - SPYT Cluster version: 3.0.1-1.13.2+yandex
2021-08-28 20:59:34,941 - INFO - spyt.client - SPYT library version: 1.3.5


# Samples

#### general dataset

In [26]:
target_spdf = spark.read.yt(TARGET_PATH)
yc_cons_spdf = spark.read.yt(FEATURES_YC_CONS)
vm_cube_spdf = spark.read.yt(FEATURES_VM_CUBE)

general_dts = (
    target_spdf
    .join(yc_cons_spdf, on =["billing_account_id", "billing_record_msk_date"])
    .join(vm_cube_spdf, on =["billing_account_id", "billing_record_msk_date"])
    .withColumn("billing_record_msk_date", F.to_date("billing_record_msk_date"))
    .select(
        "*",
        F.floor(F.datediff(col("billing_record_msk_date"), lit(period_start))/7)
            .cast("int")
            .alias("br_week_num"),
        F.dayofweek("billing_record_msk_date").alias("br_week_day")
    )
)

general_dts.printSchema()

root
 |-- billing_account_id: string (nullable = true)
 |-- billing_record_msk_date: date (nullable = true)
 |-- days_not_used: long (nullable = true)
 |-- prev_1d_cons: double (nullable = true)
 |-- prev_7d_cons: double (nullable = true)
 |-- prev_14d_cons: double (nullable = true)
 |-- prev_21d_cons: double (nullable = true)
 |-- prev_30d_cons: double (nullable = true)
 |-- prev_45d_cons: double (nullable = true)
 |-- next_1d_cons: double (nullable = true)
 |-- next_7d_cons: double (nullable = true)
 |-- next_14d_cons: double (nullable = true)
 |-- next_21d_cons: double (nullable = true)
 |-- next_30d_cons: double (nullable = true)
 |-- next_45d_cons: double (nullable = true)
 |-- billing_record_cost_rub: double (nullable = true)
 |-- billing_record_credit_rub: double (nullable = true)
 |-- billing_record_total_rub: double (nullable = true)
 |-- billing_account_usage_status: string (nullable = true)
 |-- billing_account_person_type: string (nullable = true)
 |-- billing_account_curre

In [27]:
ttt = (
    general_dts
    .groupby("billing_record_msk_date")
    .agg(F.sum("billing_record_total_rub").alias("billing_record_total_rub"))
    .sort("billing_record_msk_date")
    .toPandas()
)
ttt

,billing_record_msk_date,billing_record_total_rub
0,2021-04-01,6.260137e+06
1,2021-04-02,4.699875e+06
2,2021-04-03,4.485682e+06
3,2021-04-04,4.547304e+06
4,2021-04-05,4.739249e+06
5,2021-04-06,4.735596e+06
6,2021-04-07,4.857094e+06
7,2021-04-08,4.840534e+06
8,2021-04-09,4.893871e+06
9,2021-04-10,4.661356e+06


#### test2_dts

In [28]:
test2_dt = (
    general_dts
    .filter(~col("next_14d_cons").isNull())
    .agg(F.max("billing_record_msk_date"))
    .collect()[0][0]
)

test2_dt

datetime.date(2021, 6, 16)

In [29]:
test2_dts = (
    general_dts
    .filter(col("billing_record_msk_date")==test2_dt)
)

test2_dts\
    .groupby("billing_record_msk_date")\
    .agg(F.count("*").alias("count"), 
         F.sum("next_7d_cons").alias("sum"))\
    .show()

+-----------------------+-----+-------------------+
|billing_record_msk_date|count|                sum|
+-----------------------+-----+-------------------+
|             2021-06-16|94291|3.918575746646815E7|
+-----------------------+-----+-------------------+



In [30]:
if rewrite_flag:
    _ = (
        test2_dts
        .withColumn("billing_record_msk_date", F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd"))
        .write
        .yt("//home/cloud_analytics/ml/scoring/trial_to_paid/tables/samples/test2", 
            mode="overwrite")
    )

#### main_dts

In [31]:
main_dts = (
    general_dts
    .filter(col("billing_record_msk_date")<test2_dt)
)

def max_by(x, y):
    return F.expr(f'max_by({x}, {y})')

shuffling_rule = (
    main_dts
    .select(
        "billing_account_id",
        "br_week_num",
        "br_week_day",
        F.rand(seed=42).alias("random_col")
    )
    .groupby("billing_account_id", "br_week_num")
    .agg(
        max_by("br_week_day", "random_col").alias("br_week_day")
    )
)

filtered_main_dts = (
    shuffling_rule
    .join(main_dts, on=["billing_account_id", "br_week_num", "br_week_day"], how="left")
    .cache()
)

shuffling_rule.groupby("br_week_day").agg(F.count("*").alias("cnt")).sort("br_week_day").show()

+-----------+------+
|br_week_day|   cnt|
+-----------+------+
|          1|139972|
|          2|141113|
|          3|142697|
|          4|128549|
|          5|139036|
|          6|139455|
|          7|139647|
+-----------+------+



In [32]:
all_bas = (
    filtered_main_dts
    .select("billing_account_id")
    .distinct()
    .select(
        "billing_account_id",
        F.rand(seed=42).alias("random_col")
    )
)

tr_bas = all_bas.filter(col("random_col")<=0.7)
ntr_bas = all_bas.filter(col("random_col")>0.7)

print("tr", tr_bas.count())

print("ntr", ntr_bas.count())

tr 66189
ntr 27934


In [33]:
num_weeks = main_dts.agg(F.max("br_week_num")).collect()[0][0]
oot_border = int(num_weeks*0.7)

print('Averall weeks count:', num_weeks)
print('OOT-border start from week num:', oot_border)

Averall weeks count: 10
OOT-border start from week num: 7


In [34]:
filtered_main_dts.groupby("br_week_num").count().sort("br_week_num").show(50)

+-----------+-----+
|br_week_num|count|
+-----------+-----+
|          0|82420|
|          1|83464|
|          2|84890|
|          3|86206|
|          4|87153|
|          5|87946|
|          6|89056|
|          7|90377|
|          8|91565|
|          9|93269|
|         10|94123|
+-----------+-----+



#### sampling

In [35]:
train_base = (
    filtered_main_dts.
    join(tr_bas[["billing_account_id"]], on=["billing_account_id"], how="right")
)

test_base = (
    filtered_main_dts.
    join(ntr_bas[["billing_account_id"]], on=["billing_account_id"], how="right")
)

train = (
    train_base
    .filter(col("br_week_num")<=oot_border)
)

oot_tr = (
    train_base
    .filter(col("br_week_num")>oot_border)
    .filter(col("br_week_num")<num_weeks)
)

test = (
    test_base
    .filter(col("br_week_num")<=oot_border)
)

oot_te = (
    test_base
    .filter(col("br_week_num")>oot_border)
    .filter(col("br_week_num")<num_weeks)
)

In [36]:
if rewrite_flag:
    _ = (
        train
        .withColumn("billing_record_msk_date", F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd"))
        .write
        .yt("//home/cloud_analytics/ml/scoring/trial_to_paid/tables/samples/train", 
            mode="overwrite")
    )
    
    _ = (
        test
        .withColumn("billing_record_msk_date", F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd"))
        .write
        .yt("//home/cloud_analytics/ml/scoring/trial_to_paid/tables/samples/test", 
            mode="overwrite")
    )
    
    _ = (
        oot_te
        .withColumn("billing_record_msk_date", F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd"))
        .write
        .yt("//home/cloud_analytics/ml/scoring/trial_to_paid/tables/samples/oot", 
            mode="overwrite")
    )
    
    _ = (
        oot_tr
        .withColumn("billing_record_msk_date", F.date_format(col("billing_record_msk_date"),"yyyy-MM-dd"))
        .write
        .yt("//home/cloud_analytics/ml/scoring/trial_to_paid/tables/samples/oot_tr", 
            mode="overwrite")
    )

In [37]:
spyt.stop(spark)